In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/helen-eye-dataset")

print("Path to dataset files:", path)


ModuleNotFoundError: No module named 'kagglehub'

In [11]:
pip install kagglehub

   ---------------------------------------- 0.0/68.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.3 kB ? eta -:--:--
   ---------------------------------------- 68.3/68.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/helen-eye-dataset")

print("Path to dataset files:", path)



c:\Users\Adarsh Sharma\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 438M/438M [03:08<00:00, 2.44MB/s] 

Extracting files...


Path to dataset files: C:\Users\Adarsh Sharma\.cache\kagglehub\datasets\kmader\helen-eye-dataset\versions\2


In [3]:
# Fix MediaPipe DLL issue on Windows
# First, uninstall and reinstall MediaPipe
%pip uninstall -y mediapipe
%pip install --upgrade pip
%pip install mediapipe==0.10.9 opencv-python

print("MediaPipe reinstalled. If you still get DLL errors, please:")
print("1. Download and install Visual C++ Redistributables from:")
print("   https://aka.ms/vs/17/release/vc_redist.x64.exe")
print("2. Restart your kernel after installation")


Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
MediaPipe reinstalled. If you still get DLL errors, please:
1. Download and install Visual C++ Redistributables from:
   https://aka.ms/vs/17/release/vc_redist.x64.exe
2. Restart your kernel after installation


ERROR: Could not find a version that satisfies the requirement mediapipe==0.10.9 (from versions: 0.10.13, 0.10.14, 0.10.18, 0.10.20, 0.10.21)
ERROR: No matching distribution found for mediapipe==0.10.9


Defaulting to user installation because normal site-packages is not writeable


In [5]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

mp_face_mesh = mp.solutions.face_mesh


image = cv2.imread("C:/Users/Adarsh Sharma/OneDrive/Desktop/MAJOR/archive/train_1/train_1/1629243_1.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True
) as face_mesh:

    results = face_mesh.process(image_rgb)

annotated = image_rgb.copy()

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        for lm in face_landmarks.landmark:
            h, w, _ = annotated.shape
            x, y = int(lm.x * w), int(lm.y * h)
            cv2.circle(annotated, (x, y), 1, (0, 255, 0), -1)

plt.imshow(annotated)
plt.title("Face Mesh Visualization")
plt.axis("off")
plt.show()

ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.